In [6]:
import re  # Для препроцессинга
!pip install pandas
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [7]:
# Считываю файл
data = pd.read_csv('Сlean.csv')
df=data['description']
df.shape

(100723,)

In [8]:
df.head()

0    Прекрасная однокомнатная квартира в доме с лиф...
1    Светлая, просторная комната 19 кв. метров. Отд...
2    Первомайская, 15 мин. пешкомТёплая уютная квар...
3    в шаговой доступности парк Сокольники. , Разви...
4    Ильинское с.Все коммуникации центральные.Кварт...
Name: description, dtype: object

In [9]:
df.isnull().sum()

0

In [10]:
# Загружаю модель
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 15.3/15.3 MB 8.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [11]:
nlp = spacy.load('ru_core_news_sm', disable=['ner', 'parser']) # отключение распознавания именованных объектов для повышения скорости
def cleaning(doc):
    #Лемматизируется и удаляет стоп-слова
    # doc должен быть пространственным объектом Doc
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec использует контекстные слова для изучения векторного представления целевого слова,
    # если предложение состоит всего из одного или двух слов,
    # польза от тренинга мала, поэтому
    if len(txt) > 2:
        return ' '.join(txt)

INFO - 03:05:14: Loading dictionaries from c:\Users\Daniel\anaconda3\lib\site-packages\pymorphy2_dicts_ru\data
INFO - 03:05:14: format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [12]:
#Удаляю неалфавитные символы:
brief_cleaning = (re.sub("[^А-Яа-я]", ' ', str(row)).lower() for row in data['description'])

In [9]:
# Атрибут space .pipe() для ускорения процесса очистки:
# Подбираю оптимальное batch_size=2000, n_process=4
t = time()
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=2000, n_process=4)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 26.03 mins


In [10]:
#Помещаю результаты в фрейм данных, чтобы удалить недостающие значения и дубликаты:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(65113, 1)

In [11]:
df_clean.head(20)

,clean
0,прекрасный однокомнатный квартира дом лифт мус...
1,светлый просторный комната кв метр о...
2,первомайский мина пешкомт плая уютный к...
3,шаговый доступность парк сокольники развит...
4,ильинский коммуникация центральный квартира хо...
5,продаваться срочно однокомнатный квартира ремо...
6,знамя октябрь пос продаваться светлый уютный...
7,маяковская мина пешкомбаррикадная ...
8,продаваться двухкомнатный квартира общий площа...
9,домодедово успейте купить выгодно ...


In [30]:
df_clean.to_csv('PRE.csv',index=True)

In [13]:
df_clean = pd.read_csv('E:\project university\Выпускная работа\PRE.csv')

In [14]:
# использую пакет фраз Gensim для автоматического определения общих фраз (биграмм) из списка предложений.
from gensim.models.phrases import Phrases, Phraser

In [15]:
# Phrases() принимает список слов в качестве входных данных:
sent = [row.split() for row in df_clean['clean']]

In [16]:
# фразы из списка предложений:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 03:07:02: collecting all words and their counts
INFO - 03:07:02: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 03:07:03: PROGRESS: at sentence #10000, processed 623358 words and 173825 word types
INFO - 03:07:04: PROGRESS: at sentence #20000, processed 1187655 words and 273413 word types
INFO - 03:07:05: PROGRESS: at sentence #30000, processed 1733814 words and 343709 word types
INFO - 03:07:07: PROGRESS: at sentence #40000, processed 2866897 words and 443755 word types
INFO - 03:07:09: PROGRESS: at sentence #50000, processed 3884567 words and 528139 word types
INFO - 03:07:11: PROGRESS: at sentence #60000, processed 4895880 words and 598383 word types
INFO - 03:07:11: collected 635089 token types (unigram + bigrams) from a corpus of 5313725 words and 65113 sentences
INFO - 03:07:11: merged Phrases<635089 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 03:07:11: Phrases lifecycle event {'msg': 'built Phrases<635089 vocab, min_count=30, 

In [17]:
# Phrase() - сокращает потребление памяти Phrases(), отбросив состояние модели, которое строго не требуется для задачи обнаружения биграмм:
bigram = Phraser(phrases)

INFO - 03:07:13: exporting phrases from Phrases<635089 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 03:07:15: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<3862 phrases, min_count=30, threshold=10.0> from Phrases<635089 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 2.15s', 'datetime': '2022-11-17T03:07:15.891541', 'gensim': '4.2.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [18]:
#Преобразование корпуса на основе обнаруженных биграмм:
sentences = bigram[sent]

In [19]:
#проверка эффективности лемматизации, удаление стоп-слов и добавление биграмм.
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

46478

In [20]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['квартира',
 'дом',
 'кв',
 'этаж',
 'двор',
 'район',
 'школа',
 'комплекс',
 'жилой',
 'кухня']

In [21]:
#использую реализацию Gensim word2vec
import multiprocessing
from gensim.models import Word2Vec

In [22]:
cores = multiprocessing.cpu_count() # Количество ядер на компе

In [23]:
#Параметры модели
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

INFO - 03:07:36: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2022-11-17T03:07:36.757767', 'gensim': '4.2.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [24]:
# Word2Vec требует создания таблицы словарного запаса 
# (просто переварив все слова и отфильтровав уникальные слова, а также выполнив некоторые базовые подсчеты по ним):
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 03:07:39: collecting all words and their counts
INFO - 03:07:39: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 03:07:40: PROGRESS: at sentence #10000, processed 519503 words, keeping 17344 word types
INFO - 03:07:41: PROGRESS: at sentence #20000, processed 990655 words, keeping 23260 word types
INFO - 03:07:42: PROGRESS: at sentence #30000, processed 1449491 words, keeping 27016 word types
INFO - 03:07:43: PROGRESS: at sentence #40000, processed 2315237 words, keeping 34080 word types
INFO - 03:07:45: PROGRESS: at sentence #50000, processed 3117085 words, keeping 39644 word types
INFO - 03:07:46: PROGRESS: at sentence #60000, processed 3902562 words, keeping 44006 word types
INFO - 03:07:47: collected 46478 word types from a corpus of 4245081 raw words and 65113 sentences
INFO - 03:07:47: Creating a fresh vocabulary
INFO - 03:07:47: Word2Vec lifecycle event {'msg': 'effective_min_count=20 retains 9666 unique words (20.80% of original 46478, drops 36812

Time to build vocab: 0.13 mins


In [25]:
#Обучение модели
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 03:07:50: Word2Vec lifecycle event {'msg': 'training model with 11 workers on 9666 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2 shrink_windows=True', 'datetime': '2022-11-17T03:07:50.771456', 'gensim': '4.2.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'train'}
INFO - 03:07:51: EPOCH 0 - PROGRESS: at 7.98% examples, 104376 words/s, in_qsize 21, out_qsize 1
INFO - 03:07:52: EPOCH 0 - PROGRESS: at 19.86% examples, 132315 words/s, in_qsize 21, out_qsize 0
INFO - 03:07:54: EPOCH 0 - PROGRESS: at 36.95% examples, 152781 words/s, in_qsize 22, out_qsize 0
INFO - 03:07:55: EPOCH 0 - PROGRESS: at 49.64% examples, 166696 words/s, in_qsize 15, out_qsize 1
INFO - 03:07:56: EPOCH 0 - PROGRESS: at 56.69% examples, 168008 words/s, in_qsize 22, out_qsize 0
INFO - 03:07:57: EPOCH 0 - PROGRESS: at 66.04% examples, 173679 words/s, in_qsize 21, out_qsize 0
INFO - 03:07:58: EPOCH 0

Time to train the model: 4.91 mins


In [26]:
#Поскольку не планирую дальше обучать модель, вызываю init_sims(), что сделает модель намного более эффективной с точки зрения использования памяти:
w2v_model.init_sims(replace=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_4256\514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
WARNING - 03:12:49: destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


In [27]:
#Наиболее похожие слова на:
w2v_model.wv.most_similar(positive=["добираться"])

[('добраться', 0.5699493288993835),
 ('доехать', 0.5515435934066772),
 ('быстро_добраться', 0.5373855829238892),
 ('добраться_точка', 0.5255565643310547),
 ('легко_добраться', 0.5173808336257935),
 ('выехать', 0.48957493901252747),
 ('доезжать', 0.4788176119327545),
 ('быстро_добираться', 0.47464847564697266),
 ('передвигаться', 0.4699220657348633),
 ('минута', 0.4339616298675537)]

In [32]:
w2v_model.wv.most_similar(positive=["живописный"])

[('река', 0.5106480121612549),
 ('красивейший', 0.50239098072052),
 ('парк', 0.48951268196105957),
 ('красивый', 0.45582449436187744),
 ('прекрасный', 0.4526798129081726),
 ('зелёный', 0.4491797685623169),
 ('москва', 0.44286859035491943),
 ('благоустроенный', 0.43679338693618774),
 ('строгинский', 0.4306597113609314),
 ('иваньковский', 0.4296247661113739)]

In [28]:
#Сходство слов
w2v_model.wv.similarity("дом", 'год',)

0.25797534

In [37]:
#Сходство слов
w2v_model.wv.similarity("ремонт", 'дорога',)

-0.1590653

In [29]:
#Лишнее слово
w2v_model.wv.doesnt_match(['квартира', 'кухня', 'метро'])

'метро'

In [38]:
#Лишнее слово
w2v_model.wv.doesnt_match(['транспорт', 'этаж', 'парк'])

'этаж'

In [45]:
#Значимость слов 
w2v_model.wv.most_similar(positive=['квартира', 'этаж'], negative=['транспорт'], topn=3)

[('общий_площадь', 0.5650015473365784),
 ('кв', 0.5543716549873352),
 ('просторный', 0.5476530194282532)]

In [46]:
w2v_model.wv.index_to_key

['квартира',
 'дом',
 'кв',
 'этаж',
 'двор',
 'район',
 'школа',
 'комплекс',
 'жилой',
 'кухня',
 'москва',
 'ремонт',
 'метро',
 'два',
 'детский_сад',
 'парк',
 'жк',
 'комната',
 'детский',
 'окно',
 'отделка',
 'новый',
 'просторный',
 'находиться',
 'год',
 'минута',
 'продаваться',
 'корпус',
 'хороший',
 'сделка',
 'шаговый_доступность',
 'территория',
 'станция_метро',
 'инфраструктура',
 'центр',
 'магазин',
 'студия',
 'площадь',
 'санузел',
 'комнатный',
 'квартал',
 'продажа',
 'площадка',
 'свободный_продажа',
 'взрослый_собственник',
 'общий_площадь',
 'отличный',
 'минута_пешком',
 'собственник',
 'зона',
 'современный',
 'больший',
 'место',
 'ипотека',
 'планировка',
 'поликлиника',
 'пеший_доступность',
 'уютный',
 'вид',
 'лоджия',
 'светлый',
 'подъезд',
 'подземный_паркинг',
 'спальня',
 'расположен',
 'отдых',
 'балкон',
 'собственность',
 'гостиная',
 'окно_выходить',
 'развитой_инфраструктура',
 'высота_потолок',
 'сторона',
 'бизнес_класс',
 'сделать',
 'боль